In [13]:
import collections
import re
import sys
import warnings
from pathlib import Path
from pprint import pprint
import pandas as pd
import nibabel as nb
import numpy as np
import scipy.io

import matplotlib
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from mne_bids.tsv_handler import _from_tsv
sys.path.append("../../")

In [4]:
def visualize_electrodes(img, clusters, radius, threshold, output_fpath):
    import matplotlib
    import matplotlib.pyplot as plt
    from mpl_toolkits.mplot3d import Axes3D

    # get limits based on the image shape in voxels
    xlim, ylim, zlim = img.shape
    axcodes = nb.aff2axcodes(img.affine)

    # rotate the axes and update
    angles = [0, 45, 90, 180, 270, 360]

    # create figure
    fig, axs = plt.subplots(3, 2, figsize=(8,8))

    # generate a rotated figure for each plot
    for i, angle in enumerate(angles):
        ax = axs.flat[i]
        # ax = Axes3D(fig)
        ax.view_init(azim=angle)
        # label_colors = ['b', 'g', 'r', 'c', 'm', 'y', 'k', '0.5']
        for i, electrode in enumerate(clusters):
            clusters_i = []
            clusters_j = []
            clusters_k = []
            for cluster_id in clusters[electrode]:
                for point in clusters[electrode][cluster_id]:
                    clusters_i.append(point[0])
                    clusters_j.append(point[1])
                    clusters_k.append(point[2])
            ax.scatter3D(clusters_i, clusters_j, clusters_k, label=electrode)
        ax.set_xlim3d(0, xlim+1)
        ax.set_ylim3d(0, ylim+1)
        ax.set_zlim3d(0, zlim+1)
        ax.set_title(f'Azim-Angled {angle} Post-Processing Contacts \n'
                     '(Radius = %d voxels, Threshold = %.3f)' % (radius, threshold))
        ax.set_xlabel(f'(x) {axcodes[0]}')
        ax.set_ylabel(f'(x) {axcodes[1]}')
        ax.set_zlabel(f'(x) {axcodes[2]}')
        ax.legend(loc='upper center', bbox_to_anchor=(0.2, 0.8), shadow=True, ncol=1);

    fig.tight_layout()
    plt.savefig(output_fpath, box_inches="tight")
    plt.close(fig)

# Load in Clustered Voxels

In [7]:
data_dict = scipy.io.loadmat('/Users/adam2392/Downloads/neuroimg/derivatives/freesurfer/la03/elecs/la03_clusteredelec_voxels.mat')
ch_names = data_dict['eleclabels']
voxel_coords = data_dict['elecmatrix']

print(data_dict.keys())

dict_keys(['__header__', '__version__', '__globals__', 'eleclabels', 'elecmatrix'])


In [9]:
print(ch_names.shape)
print(voxel_coords.shape)

(185, 3)
(185, 3)


# Load in Electrodes TSV file

In [78]:
def find_wmchs_in_electrodestsv(fname, atlas_name=None):
    if atlas_name is None:
        atlas_name = ['desikan-killiany', 'destriuex']
    else:
        atlas_name = [atlas_name]
        
    electrodes_tsv = _from_tsv(fname)
    electrodes_df = pd.DataFrame.from_dict(electrodes_tsv)

    atlas_wm_chs = []
    for atlas in atlas_name:
        # get wm from destrieux/dk atlases
        electrodes_df[atlas] = electrodes_df[atlas].str.lower()
        atlas_wm = electrodes_df[electrodes_df[atlas].str.contains("white-matter")]
        wm_chs = atlas_wm['name'].tolist()
        atlas_wm_chs.append(wm_chs)
        print(len(wm_chs))
    
    all_wm_chs = atlas_wm_chs
    wm_chs = set().union(*all_wm_chs)
    wm_chs = [x for x in wm_chs if not x.upper().startswith("L")]
    return wm_chs

In [79]:
# get wm from destrieux
electrodes_df['desikan-killiany'] = electrodes_df['desikan-killiany'].str.lower()
electrodes_df['destriuex'] = electrodes_df['destriuex'].str.lower()

destrieux_wm = electrodes_df[electrodes_df['destriuex'].str.contains("white-matter")]
dest_wm_chs = destrieux_wm['name'].tolist()

dk_wm = electrodes_df[electrodes_df['desikan-killiany'].str.contains("white-matter")]
dk_wm_chs = dk_wm['name'].tolist()

np.array_equal(dk_wm_chs, dest_wm_chs)
print(set(dest_wm_chs).difference(set(dk_wm_chs)))
print(len(dk_wm_chs))

set()
72


In [80]:
bidsroot = Path("/Users/adam2392/Dropbox/epilepsy_bids/")
derivdir = Path(bidsroot / 'derivatives' / 'freesurfer')
electrodes_fname = Path(bidsroot / 'sub-la03' / 'ses-veeg' / "sub-la03_ses-veeg_acq-seeg_proc-manual_electrodes.tsv")

# use function to find 
wm_chs = find_wmchs_in_electrodestsv(electrodes_fname, atlas_name=None)

# load tsv and df into RAM
electrodes_tsv = _from_tsv(electrodes_fname)
electrodes_df = pd.DataFrame.from_dict(electrodes_tsv)

print(len(wm_chs))
print(wm_chs)

pd.set_option('display.max_rows', None)
display(electrodes_df)

72
72
63
['F5', 'X8', 'R4', 'W11', 'X7', 'X9', 'C6', 'R5', 'C15', 'W12', 'X6', 'V13', 'Z12', 'Z2', 'C14', 'Z4', 'Z7', 'U1', 'Y11', 'S7', 'S8', 'P9', 'R6', 'Z11', 'W10', 'V14', 'X5', 'C13', 'V12', 'F10', 'S9', 'Z6', 'R8', 'S1', 'W14', 'C9', 'U2', 'F7', 'P14', 'C7', 'F4', 'W5', 'O2', 'F8', 'V7', 'Z13', 'Z3', 'P8', 'V8', 'V2', 'Z10', 'W4', 'R1', 'X10', 'Y3', 'R2', 'C8', 'F9', 'W9', 'W3', 'V11', 'Z5', 'W6']


,name,x,y,z,size,destriuex,desikan-killiany
0,P1,-1.87339,-22.0596,78.9137,n/a,Unknown,Unknown
1,P2,2.14987,-21.6754,78.3847,n/a,ctx_rh_G_and_S_paracentral,ctx-rh-paracentral
2,P3,6.06139,-21.5402,78.5495,n/a,ctx_rh_S_cingul-Marginalis,ctx-rh-paracentral
3,P8,26.3207,-21.0628,78.8775,n/a,Right-Cerebral-White-Matter,Right-Cerebral-White-Matter
4,P11,38.3988,-21.0688,79.5156,n/a,ctx_rh_S_postcentral,ctx-rh-postcentral
5,P12,42.6454,-21.3457,79.6808,n/a,ctx_rh_G_postcentral,ctx-rh-postcentral
6,P13,46.574,-21.2114,79.91,n/a,ctx_rh_G_postcentral,ctx-rh-postcentral
7,P14,50.8417,-20.8784,79.3559,n/a,Right-Cerebral-White-Matter,Right-Cerebral-White-Matter
8,P15,55.1192,-20.4045,79.7939,n/a,Unknown,Unknown
9,W1,-3.1894,-47.3354,73.9111,n/a,Unknown,Unknown


In [81]:
electrodes_fname = Path(bidsroot / 'sub-la05' / 'ses-veeg' / "sub-la05_ses-veeg_acq-seeg_proc-manual_electrodes.tsv")

# use function to find 
wm_chs = find_wmchs_in_electrodestsv(electrodes_fname, atlas_name=None)

# load tsv and df into RAM
electrodes_tsv = _from_tsv(electrodes_fname)
electrodes_df = pd.DataFrame.from_dict(electrodes_tsv)

print(len(wm_chs))
display(electrodes_df)

72
72
72


,name,x,y,z,size,destriuex,desikan-killiany
0,X'1,29.9262,-22.1254,41.2848,n/a,Right-Cerebral-White-Matter,Right-Cerebral-White-Matter
1,X'2,33.0394,-22.4952,41.1243,n/a,Right-Cerebral-White-Matter,Right-Cerebral-White-Matter
2,X'10,61.4804,-22.3307,39.0162,n/a,ctx_rh_G_temporal_middle,ctx-rh-middletemporal
3,X'11,66.2279,-21.8092,38.9681,n/a,Unknown,Unknown
4,S'1,0.367546,-16.1985,54.8767,n/a,CSF,CSF
5,S'2,4.5369,-15.7588,55.6999,n/a,Right-Thalamus-Proper,Right-Thalamus-Proper
6,S'3,8.85795,-16.1215,55.6933,n/a,Right-Thalamus-Proper,Right-Thalamus-Proper
7,S'4,13.4938,-16.0582,55.5746,n/a,Right-Thalamus-Proper,Right-Thalamus-Proper
8,S'12,49.0232,-16.3063,54.2704,n/a,Right-Cerebral-White-Matter,Right-Cerebral-White-Matter
9,S'13,53.2923,-16.2198,54.1065,n/a,ctx_rh_G_temp_sup-Lateral,ctx-rh-superiortemporal
